# Tabular Data

In [1]:
import pickle
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, dodge
from math import pi
from bokeh.layouts import column, row, grid, gridplot
from bokeh.palettes import Set3
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

In [2]:
from bokeh.io import output_notebook

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
with open("2022testdata-qm20.pickle", "rb") as pfile:
    data = pickle.load(pfile)

measures = data['measures'] 
matches = data['matches']

# finds last match in measures
last_row = measures.iloc[-1]
lm = last_row[0]
last_match = int(lm.split('m')[1])
# filters matches to qm matches only and converts to numbered matches
qmmatch = matches[matches.match.str.startswith('qm') == True].copy()
new = qmmatch['match'].str.split('m', expand = True)
qmmatch['match_number'] = new[1]
qmmatch['match_number'] = pd.to_numeric(qmmatch['match_number'])

# filters qmmatches to matches before 20
lmatches = qmmatch[qmmatch.match_number < last_match]
lmatches

matches_played = (lmatches.groupby('team_number')
                  .size()
                  .reset_index()
                  .set_index('team_number')
                  .rename({0: 'matches_played'}, axis='columns')
                 )

measures2 = measures[measures.team_number.str.startswith('frc') == True]
measures2 = measures2[measures2.task == 'start_pos']

# pichart has position values
pichart = (
    measures2.groupby(['team_number', 'measure1'])
    .size()
    .unstack("measure1")
    .merge(matches_played, on='team_number')
)

In [5]:
pichart

,center,left,right,matches_played
team_number,,,,
frc1294,1.0,NaN,2.0,3
frc1318,NaN,1.0,2.0,3
frc1778,1.0,NaN,1.0,3
frc1899,2.0,NaN,2.0,3
frc2412,1.0,NaN,3.0,3
frc2522,NaN,1.0,2.0,3
frc2903,2.0,NaN,2.0,3
frc2910,NaN,3.0,NaN,3
frc2928,NaN,NaN,4.0,4


In [6]:
measures

,match,team_number,phase,task,measure1,measure2,measure_type
0,qm1,frc4683,auto,start_pos,center,-1,categorical
1,qm1,frc4131,auto,start_pos,left,-1,categorical
2,qm1,frc4131,auto,taxi,true,-1,boolean
3,qm1,frc4683,auto,taxi,true,-1,boolean
4,qm1,frc4131,auto,start_cargo,true,-1,boolean
...,...,...,...,...,...,...,...
1160,qm20,frc4180,tele,lower,5,4,count
1161,qm20,frc2412,tele,pickup-field,7,-1,count
1162,qm20,frc2412,endgame,hangar_level_start,2,-1,count
1163,qm20,frc2412,endgame,hangar_level_end,3,-1,count


In [51]:
melted_measures = measures.melt(
            id_vars=["match", "team_number", "phase", "task", "measure_type"],
            value_vars=["measure1", "measure2"],
            var_name="measure_col")
        
counts = melted_measures[melted_measures.measure_type == "count"].copy()
counts["value"] = pd.to_numeric(counts.value)
counts["value"] = counts.value.replace(-1, np.nan)

means = counts.pivot_table(
    "value",
    index=["team_number", "task", "measure_col"],
    columns=["phase"])
means = means.fillna(0)

filtered = means.unstack(["task", "measure_col"])

In [52]:
pichart

,center,left,right,matches_played
team_number,,,,
frc1294,1.0,NaN,2.0,3
frc1318,NaN,1.0,2.0,3
frc1778,1.0,NaN,1.0,3
frc1899,2.0,NaN,2.0,3
frc2412,1.0,NaN,3.0,3
frc2522,NaN,1.0,2.0,3
frc2903,2.0,NaN,2.0,3
frc2910,NaN,3.0,NaN,3
frc2928,NaN,NaN,4.0,4


In [53]:
# filtered.columns = filtered.columns.get_level_values(0) + '_' +  filtered.columns.get_level_values(1) + '_' + filtered.columns.get_level_values(2)

In [54]:
nan_value = float("NaN")
filtered = filtered.replace(0, nan_value)
filtered = filtered.dropna(how='all', axis=1)

In [55]:
# filtered = filtered.merge(pichart, on='team_number')

In [56]:
# filtered['center']

# Autonomous Table

In [57]:
measures

,match,team_number,phase,task,measure1,measure2,measure_type
0,qm1,frc4683,auto,start_pos,center,-1,categorical
1,qm1,frc4131,auto,start_pos,left,-1,categorical
2,qm1,frc4131,auto,taxi,true,-1,boolean
3,qm1,frc4683,auto,taxi,true,-1,boolean
4,qm1,frc4131,auto,start_cargo,true,-1,boolean
...,...,...,...,...,...,...,...
1160,qm20,frc4180,tele,lower,5,4,count
1161,qm20,frc2412,tele,pickup-field,7,-1,count
1162,qm20,frc2412,endgame,hangar_level_start,2,-1,count
1163,qm20,frc2412,endgame,hangar_level_end,3,-1,count


In [58]:
filtered

phase            auto                                                          \
task               hp               lower           pickup_terminal     upper   
measure_col  measure1  measure2  measure1  measure2        measure1  measure1   
team_number                                                                     
frc1294      1.000000       NaN  0.333333  0.666667        2.000000       NaN   
frc1318           NaN  0.500000       NaN       NaN             NaN       NaN   
frc1778           NaN  1.000000       NaN  0.500000             NaN       NaN   
frc1899           NaN  1.000000  1.000000  0.333333        1.000000       NaN   
frc2412           NaN  1.000000       NaN       NaN        0.666667  1.750000   
frc2522           NaN  1.000000  0.500000  1.000000        1.000000       NaN   
frc2903           NaN       NaN       NaN       NaN             NaN  0.500000   
frc2910      0.333333  0.666667       NaN  1.000000        2.000000  1.000000   
frc2928           NaN       NaN  0.666667  0.333333             NaN       NaN   
frc2930      1.000000       NaN       NaN       NaN             NaN       NaN   
frc2976           NaN  1.000000       NaN  1.000000             NaN       NaN   
frc3070           NaN       NaN       NaN  1.000000             NaN       NaN   
frc3268           NaN  1.000000  1.333333  0.666667        0.500000       NaN   
frc3826      1.000000       NaN       NaN       NaN        2.000000       NaN   
frc4089      1.000000       NaN  0.333333  1.666667        2.000000  1.000000   
frc4131      0.333333  0.666667       NaN       NaN        1.500000  0.666667   
frc4173           NaN       NaN       NaN       NaN             NaN       NaN   
frc4180           NaN       NaN  0.250000  1.000000        1.500000  0.333333   
frc4205      1.000000       NaN       NaN  1.333333        1.000000       NaN   
frc4309           NaN       NaN       NaN  1.000000             NaN       NaN   
frc4512      1.000000       NaN       NaN       NaN             NaN       NaN   
frc4513           NaN       NaN       NaN  1.000000             NaN       NaN   
frc4681      1.000000       NaN  1.000000  0.500000             NaN       NaN   
frc4682           NaN  1.000000       NaN       NaN             NaN  1.000000   
frc4683           NaN       NaN       NaN       NaN             NaN       NaN   
frc4911      0.500000  0.500000       NaN       NaN             NaN  0.333333   
frc4915           NaN       NaN       NaN       NaN             NaN  1.333333   
frc4918      1.000000       NaN       NaN       NaN        2.000000  1.750000   
frc492       1.000000       NaN       NaN  1.000000             NaN  0.333333   
frc5588      0.500000  0.500000       NaN       NaN        1.000000  2.000000   
frc7118           NaN  1.000000       NaN  1.000000             NaN       NaN   
frc7461           NaN       NaN  0.500000  1.000000        2.000000       NaN   
frc7627           NaN       NaN       NaN  1.000000             NaN       NaN   
frc8032           NaN       NaN  1.000000       NaN        1.000000       NaN   
frc8059      1.000000       NaN       NaN       NaN             NaN  0.666667   
frc8248      1.000000       NaN  1.000000       NaN             NaN  1.666667   
frc949       0.666667  0.333333  0.666667  0.666667             NaN  1.000000   

phase                                       endgame                     \
task                  pickup_field hangar_level_end hangar_level_start   
measure_col  measure2     measure1         measure1           measure1   
team_number                                                              
frc1294           NaN          NaN              NaN                NaN   
frc1318      2.500000     1.500000         2.000000           2.000000   
frc1778           NaN          NaN              NaN                NaN   
frc1899           NaN          NaN              NaN                NaN   
frc2412      1.000000     1.250000         2.750000           2.000000   
frc2522     

In [59]:
pi

3.141592653589793

In [60]:
pichart

,center,left,right,matches_played
team_number,,,,
frc1294,1.0,NaN,2.0,3
frc1318,NaN,1.0,2.0,3
frc1778,1.0,NaN,1.0,3
frc1899,2.0,NaN,2.0,3
frc2412,1.0,NaN,3.0,3
frc2522,NaN,1.0,2.0,3
frc2903,2.0,NaN,2.0,3
frc2910,NaN,3.0,NaN,3
frc2928,NaN,NaN,4.0,4


In [63]:
from os.path import dirname, join

import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import (Button, ColumnDataSource, CustomJS, DataTable,
                          NumberFormatter, RangeSlider, TableColumn)
from bokeh.layouts import column, row, grid, gridplot
filtered = filtered.fillna(0).round(1)
source = ColumnDataSource(filtered)
source.data.keys()

dict_keys(['team_number', 'auto_hp_measure1', 'auto_hp_measure2', 'auto_lower_measure1', 'auto_lower_measure2', 'auto_pickup_terminal_measure1', 'auto_upper_measure1', 'auto_upper_measure2', 'auto_pickup_field_measure1', 'endgame_hangar_level_end_measure1', 'endgame_hangar_level_start_measure1', 'endgame_penalty_count_measure1', 'tele_lower_measure1', 'tele_lower_measure2', 'tele_pickup-field_measure1', 'tele_pickup-terminal_measure1', 'tele_upper_measure1', 'tele_upper_measure2'])

In [62]:
bobbo = ['team_number', 'auto_hp_measure1', 'auto_hp_measure2', 'auto_lower_measure1', 'auto_lower_measure2', 'auto_pickup_terminal_measure1', 
         'auto_upper_measure1', 'auto_upper_measure2', 'auto_pickup_field_measure1', 'endgame_hangar_level_end_measure1', 
         'endgame_hangar_level_start_measure1', 'endgame_penalty_count_measure1', 'tele_lower_measure1', 'tele_lower_measure2', 
         'tele_pickup-field_measure1', 'tele_pickup-terminal_measure1', 'tele_upper_measure1', 'tele_upper_measure2']

bobbo2 = []
for i in bobbo:
    if i.__contains__('measure1'):
        i = i.replace("measure1", "make")
    if i.__contains__('measure2'):
        i = i.replace("measure2", "miss")
    bobbo2.append(i)
    
auto = []
tele = []
end = []
        
auto.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))
tele.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))
end.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))

    
columns = []
for i in range(0, len(bobbo)):
    columns.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))

    if(bobbo[i].__contains__("auto")):
        auto.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))


    elif(bobbo[i].__contains__("tele")):
        tele.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))

    elif(bobbo[i].__contains__("endgame")):
        end.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))
        

auto_table = DataTable(source=source, columns=auto, width=1100)
tele_table = DataTable(source=source, columns=tele, width=1100)
end_table = DataTable(source=source, columns=end, width=1100)
column_object = column(auto_table, tele_table, end_table)
show(column_object)

In [64]:
import pickle
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, dodge
from math import pi
from bokeh.layouts import column, row, grid, gridplot
from bokeh.palettes import Set3
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
import viewer.app.data as data
from os.path import dirname, join
from bokeh.io import curdoc
from bokeh.models import (Button, ColumnDataSource, CustomJS, DataTable,
                          NumberFormatter, RangeSlider, TableColumn)

class TabularPlot:
    def __init__(self):
        self.layout = None
        self.measures = None
        self.matches = None

    def prepare_data(self, measures):
        melted_measures = measures.melt(
            id_vars=["match", "team_number", "phase", "task", "measure_type"],
            value_vars=["measure1", "measure2"],
            var_name="measure_col")
        
        counts = melted_measures[melted_measures.measure_type == "count"].copy()
        counts["value"] = pd.to_numeric(counts.value)
        counts["value"] = counts.value.replace(-1, np.nan)

        means = counts.pivot_table(
            "value",
            index=["team_number", "task", "measure_col"],
            columns=["phase"])
        means = means.fillna(0)

        filtered = means.unstack(["task", "measure_col"])
        nan_value = float("NaN")
        filtered = filtered.replace(0, nan_value)
        filtered = filtered.dropna(how='all', axis=1)
        return filtered

    def graph_plot(self):
        measures = data.get_data()[0]
        filtered = self.prepare_data(measures)
        filtered = filtered.fillna(0).round(1)
        source = ColumnDataSource(filtered)

        bobbo = ['team_number', 'auto_hp_measure1', 'auto_hp_measure2', 'auto_lower_measure1', 'auto_lower_measure2', 'auto_pickup_terminal_measure1', 
         'auto_upper_measure1', 'auto_upper_measure2', 'auto_pickup_field_measure1', 'endgame_hangar_level_end_measure1', 
         'endgame_hangar_level_start_measure1', 'endgame_penalty_count_measure1', 'tele_lower_measure1', 'tele_lower_measure2', 
         'tele_pickup-field_measure1', 'tele_pickup-terminal_measure1', 'tele_upper_measure1', 'tele_upper_measure2']

        bobbo2 = []
        for i in bobbo:
            if i.__contains__('measure1'):
                i = i.replace("measure1", "make")
            if i.__contains__('measure2'):
                i = i.replace("measure2", "miss")
            bobbo2.append(i)
            
        auto = []
        tele = []
        end = []
                
        auto.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))
        tele.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))
        end.append(TableColumn(field = bobbo[0], title = bobbo2[0].replace("_", " ")))

            
        columns = []
        for i in range(0, len(bobbo)):
            columns.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))

            if(bobbo[i].__contains__("auto")):
                auto.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))


            elif(bobbo[i].__contains__("tele")):
                tele.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))

            elif(bobbo[i].__contains__("endgame")):
                end.append(TableColumn(field = bobbo[i], title = bobbo2[i].replace("_", " ")))
                

        auto_table = DataTable(source=source, columns=auto, width=1100)
        tele_table = DataTable(source=source, columns=tele, width=1100)
        end_table = DataTable(source=source, columns=end, width=1100)

        return column(auto_table, tele_table, end_table) 
    
    def get_layout(self):
        self.layout = self.graph_plot()
        return self.layout

ModuleNotFoundError: No module named 'viewer'